In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
#from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [9]:
# fixed parameters
block = MEGNet_block
head_mol,head_atom,head_edge = head_mol,head_atom,head_edge
clip = 2
batch_size = 32
threshold = -1.3
reuse = False
lr = 1e-4

In [3]:
# changing parameters
head = SimplyInteraction
data = '../Data/{}_data_ACSF_SOAP_atomInfo_otherInfo.pickle'
dim = 512
logLoss = True
weight = None
layer1 = 3
layer2 = 3
factor = 2
epochs = 150
aggr = 'max'
interleave = False

In [21]:
data = '../Data/{}_data_ACSF_expand_PCA.pickle'

In [22]:
prefix = 'fc_oof'

In [23]:
folds = []
for f in range(5):
    with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(f)+'.pickle', 'rb') as handle:
        folds.append(pickle.load(handle))
folds = [[Data(**d) for d in fold] for fold in folds]

In [33]:
oof_features = []
for i in range(5):
    print('\nstart fold '+str(i))
    # parpare data
    train_list = []
    val_list = []
    for j in range(5):
        if i == j:
            val_list.extend(folds[j])
        else:
            train_list.extend(folds[j])
    
    train_dl = DataLoader(train_list,batch_size,shuffle=True)
    val_dl = DataLoader(val_list,batch_size,shuffle=False)
    
    # train model
    model = GNN_multiHead_interleave_auxiliaryOnly(reuse,block,head,head_mol,head_atom,head_edge,\
                          dim,layer1,layer2,factor,**data_dict[data],aggr=aggr,interleave=interleave).to('cuda:0')
    paras = trainable_parameter(model)
    opt = RAdam(paras,lr=lr,weight_decay=1e-3)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-05)
    
    model = train_auxiliaryOnly(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                scheduler=scheduler,logLoss=logLoss,weight=weight,threshold=threshold)
    
    torch.save({'model_state_dict':model.state_dict()},\
                '../Model/'+prefix+'_fold'+str(i)+'.tar')
    
    # predict oof for each type
    for type_i in range(8):
        # load val data and type_id
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(i)+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_data = pickle.load(handle)
        test_list = [Data(**d) for d in test_data]
        test_dl = DataLoader(test_list,batch_size,shuffle=False)
        
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(i)+'_type_'+str(type_i)+'_id.pickle', 'rb') as handle:
            test_id = pickle.load(handle)
    
        # predict
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()        
    
        # join
        assert yhat.shape[0]==test_id.shape[0],'yhat and test_id should have same shape'
        oof_features.append(pd.DataFrame(np.concatenate([test_id.reshape(-1,1),yhat],1),columns=['id']+['feature_'+str(feature_i) for feature_i in range(yhat.shape[1])]))
    
    test_features = []    
    # predict test
    for type_i in range(8):
        # load val data and type_id
        with open(data.format('test').split('pickle')[0][:-1]+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_data = pickle.load(handle)
        test_list = [Data(**d) for d in test_data]
        test_dl = DataLoader(test_list,batch_size,shuffle=False)
        
        with open(data.format('test').split('pickle')[0][:-1]+'_id_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_id = pickle.load(handle)
    
        # predict
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()        
    
        # join
        assert yhat.shape[0]==test_id.shape[0],'yhat and test_id should have same shape'
        test_features.append(pd.DataFrame(np.concatenate([test_id.reshape(-1,1),yhat],1),columns=['id']+['feature_'+str(feature_i) for feature_i in range(yhat.shape[1])]))

    if i == 0:
        test_oof_df = pd.concat(test_features, ignore_index=True)
    else:
        temp_df = pd.concat(test_features, ignore_index=True)
        test_oof_df.loc[:,['feature_'+str(feature_i) for feature_i in range(yhat.shape[1])]] = test_oof_df.loc[:,['feature_'+str(feature_i) for feature_i in range(yhat.shape[1])]].values \
                                                                                            + temp_df.loc[:,['feature_'+str(feature_i) for feature_i in range(yhat.shape[1])]].values        


start fold 0

start fold 1

start fold 2

start fold 3

start fold 4


In [34]:
oof_df = pd.concat(oof_features, ignore_index=True)

In [35]:
test_oof_df.loc[:,['feature_'+str(feature_i) for feature_i in range(yhat.shape[1])]] = test_oof_df.loc[:,['feature_'+str(feature_i) for feature_i in range(yhat.shape[1])]]/5

In [36]:
oof_df.head()

,id,feature_0,feature_1,feature_2,feature_3
0,29752.0,-0.000747,-0.013000,0.027452,-0.048075
1,29761.0,-0.000739,-0.013015,0.027527,-0.048135
2,29768.0,-0.000791,-0.013033,0.027499,-0.048114
3,1570966.0,-0.000629,-0.013023,0.027519,-0.048190
4,1570969.0,-0.000504,-0.012894,0.027566,-0.048155


In [37]:
test_oof_df.head()

,id,feature_0,feature_1,feature_2,feature_3
0,4658148.0,-0.000348,-0.013759,0.027051,-0.048262
1,4658150.0,-0.000348,-0.013759,0.027051,-0.048262
2,4658152.0,-0.001041,-0.013132,0.027884,-0.048046
3,4658156.0,-0.001041,-0.013132,0.027884,-0.048046
4,4658159.0,-0.000802,-0.013159,0.027614,-0.048097


In [ ]:
test_oof_df.to_csv('../Data/test_oof_features_'+prefix,index=False)
oof_df.to_csv('../Data/train_oof_features_'+prefix,index=False)